In [1]:
qpath = r'../../' # path to quetzal here
data = r'inputs/'
import sys
sys.path.insert(0, qpath)

# import class
from quetzal.io.gtfs_reader import importer

C:\Users\rgoix\AppData\Local\Continuum\anaconda3\envs\quetzal_env\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Read GTFS

In [2]:
feed = importer.GtfsImporter(path=data + r'bilbao.zip', dist_units='m')
feed = feed.clean()
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20200310
5,num_routes,82
6,num_trips,3297
7,num_stops,498
8,num_shapes,0
9,num_frequencies,0


Frequency conversion currently work only for one specific service and date, and on one given time period.  
It computes the average headway over this time period.

### Restrict to one date and merge services

In [3]:
feed = feed.restrict(dates=['20191210'])
feed.group_services()

In [4]:
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,82
6,num_trips,3297
7,num_stops,498
8,num_shapes,0
9,num_frequencies,0


In [5]:
feed = feed.create_shapes()

In [6]:
feed.map_trips(
    feed.trips.groupby('route_id').first().trip_id.head(50)  # Map the first trip of each route
)

### Build patterns
In a non-frequency based GTFS, a route has many trips, each corresponding to a start at a specific time. It is necessary to group these trips in order to compute their headway. A group of trips is refered as a pattern.
The default method to build the patterns is by grouping trips based on their ordered list of stop_ids, without knowledge of time or duration.  
Other methods to build the patterns are available: by parent stations, or clusters. See example 4-advanced-patterns.ipynb

In [7]:
feed.build_patterns()

In [8]:
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,82
6,num_trips,3297
7,num_stops,498
8,num_shapes,124
9,num_frequencies,0


### Convert to frequencies

In [9]:
time_range = ['06:00:00', '09:00:00']  # time format must be HH:MM:SS
feed_f = feed.convert_to_frequencies(time_range=time_range)

100%|██████████| 76/76 [00:00<00:00, 109.31it/s]


In [10]:
feed_f.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,67
6,num_trips,76
7,num_stops,495
8,num_shapes,76
9,num_frequencies,76


The average headway computed is the interval length divided by the number of trip starts within the interval.
- a trip starting exactly at the start of the time range (6:00:00) is taken into account
- a trip starting exactly at the end of the time range (9:00:00) is not considered.